# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import Counter
import math

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)
'''text1='Los Angeles Kings'
    text2='Las Vegas'
    
    ve1=text_to_vector(text1)
    ve2=text_to_vector(text2)
    print(get_cosine(ve1,ve2))'''

def preProcessData1():
    #Basic preProcessing on dataframes
    #NHL Dataframe
    nhl = nhl_df[['team','W','L','year']]
    nhl.where(nhl['year'] == 2018, inplace=True)
    nhl.dropna(inplace=True)
    nhl = nhl[['team','W','L']]
    nhl.columns=['Team','Wins','Loses']
    nhl['Wins']=nhl['Wins'].str.extract('(\d+)', expand=False)
    nhl.dropna(inplace=True)
    nhl['Team']=nhl['Team'].replace(to_replace ='\*', value = '', regex = True)
    nhl.sort_values(by=['Team'], ascending=True, inplace=True)
    nhl.set_index('Team', inplace=True)
    #Cities Dataframe
    c = cities.drop(['MLB', 'NFL', 'NBA'], axis=1)
    c.columns = ['City','Population','Team']
    c['Team']=c['Team'].replace(to_replace ='\[.*', value = '', regex = True)
    c['Team']=c['Team'].replace("",np.nan)
    c['Team']=c['Team'].replace("—",np.nan)
    c.dropna(inplace=True)
    c.reset_index(drop=True, inplace=True)
    c['Team']=c['City']+' '+c['Team']
    c.set_index('Team', inplace=True)
    c.sort_values(by=['Team'], ascending=True, inplace=True)
    #c = c.drop(['City'],axis=1)
    ct=[]
    for team in nhl.index:
        flag=False
        maxc=0
        outc=np.nan
        for t,city in zip(c.index,c['City']):
            vec1,vec2 = text_to_vector(team), text_to_vector(t)
            cos = get_cosine(vec1,vec2)
            if(cos>maxc):
                maxc=cos
                outc=city
        ct.append(outc)
    nhl['City']=ct
    #HandCode some results
    nhl.loc['Anaheim Ducks']['City']='Los Angeles'
    return nhl, c

def nhl_correlation(flag=None): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    df1,df2= preProcessData1()
    df1['Win_Loss']=df1['Wins'].astype(int)/(df1['Loses'].astype(int)+df1['Wins'].astype(int))
    df1.sort_values('City')
    df2.sort_values('City')
    w_l_mean=[]
    cities=[]
    for group, frame in df1.groupby('City'):
        cities.append(group)
        w_l_mean.append(frame['Win_Loss'].mean())
    #print(w_l_mean, cities, df2['Population'].tolist())
    population_by_region = list(map(int,df2['Population'].tolist())) # pass in metropolitan area population from cities
    win_loss_by_region = w_l_mean # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    if(flag):
        return df1, df2
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [33]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [34]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import Counter
import math

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)
'''text1='Los Angeles Kings'
    text2='Las Vegas'
    ve1=text_to_vector(text1)
    ve2=text_to_vector(text2)
    print(get_cosine(ve1,ve2))'''

def preProcessData2():
    #Basic preProcessing on dataframes
    #NBA Dataframe
    nba = nba_df[['team','W','L','year']]
    nba.where(nba['year'] == 2018, inplace=True)
    nba.dropna(inplace=True)
    nba = nba[['team','W','L']]
    nba.columns=['Team','Wins','Loses']
    nba['Wins']=nba['Wins'].str.extract('(\d+)', expand=False)
    nba.dropna(inplace=True)
    nba['Team']=nba['Team'].replace(to_replace ='\*', value = '', regex = True)
    nba.sort_values(by=['Team'], ascending=True, inplace=True)
    nba.set_index('Team', inplace=True)
    #Cities Dataframe
    c = cities.drop(['MLB', 'NFL', 'NHL'], axis=1)
    c.columns = ['City','Population','Team']
    c['Team']=c['Team'].replace(to_replace ='\[.*', value = '', regex = True)
    c['Team']=c['Team'].replace("",np.nan)
    c['Team']=c['Team'].replace("—",np.nan)
    c.dropna(inplace=True)
    c.reset_index(drop=True, inplace=True)
    c['Team']=c['City']+' '+c['Team']
    c.set_index('Team', inplace=True)
    c.sort_values(by=['Team'], ascending=True, inplace=True)
    #c = c.drop(['City'],axis=1)
    ct=[]
    for team in nba.index:
        flag=False
        maxc=0
        outc=np.nan
        for t,city in zip(c.index,c['City']):
            vec1,vec2 = text_to_vector(team), text_to_vector(t)
            cos = get_cosine(vec1,vec2)
            if(cos>maxc):
                maxc=cos
                outc=city
        ct.append(outc)
    nba['City']=ct
    #HandCode some results
    nba.loc['Brooklyn Nets (12)']['City']='New York City'
    return nba, c

def nba_correlation(flag=None):
    # YOUR CODE HERE
    df11,df22= preProcessData2()
    df11['Win_Loss']=df11['Wins'].astype(int)/(df11['Loses'].astype(int)+df11['Wins'].astype(int))
    df11.sort_values('City')
    df22.sort_values('City')
    w_l_mean=[]
    cities=[]
    for group, frame in df11.groupby('City'):
        cities.append(group)
        w_l_mean.append(frame['Win_Loss'].mean())
    #raise NotImplementedError()
    population_by_region = list(map(int,df22['Population'].tolist())) # pass in metropolitan area population from cities
    win_loss_by_region = w_l_mean# pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    if(flag):
        return df11, df22
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [35]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [36]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import Counter
import math

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

WORD = re.compile(r"\w+")
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)
'''text1='Los Angeles Kings'
    text2='Las Vegas'
    ve1=text_to_vector(text1)
    ve2=text_to_vector(text2)
    print(get_cosine(ve1,ve2))'''

def preProcessData3():
    #Basic preProcessing on dataframes
    #NBA Dataframe
    mlb = mlb_df[['team','W','L','year']]
    mlb.where(mlb['year'] == 2018, inplace=True)
    mlb.dropna(inplace=True)
    mlb = mlb[['team','W','L']]
    mlb.columns=['Team','Wins','Loses']
    #mlb['Wins']=mlb['Wins'].str.extract('(\d+)', expand=False)
    mlb.dropna(inplace=True)
    mlb['Team']=mlb['Team'].replace(to_replace ='\*', value = '', regex = True)
    mlb.sort_values(by=['Team'], ascending=True, inplace=True)
    mlb.set_index('Team', inplace=True)
    #Cities Dataframe
    c = cities.drop(['NBA', 'NFL', 'NHL'], axis=1)
    c.columns = ['City','Population','Team']
    c['Team']=c['Team'].replace(to_replace ='\[.*', value = '', regex = True)
    c['Team']=c['Team'].replace("",np.nan)
    c['Team']=c['Team'].replace("—",np.nan)
    c.dropna(inplace=True)
    c.reset_index(drop=True, inplace=True)
    c['Team']=c['City']+' '+c['Team']
    c.set_index('Team', inplace=True)
    c.sort_values(by=['Team'], ascending=True, inplace=True)
    #c = c.drop(['City'],axis=1)
    ct=[]
    for team in mlb.index:
        flag=False
        maxc=0
        outc=np.nan
        for t,city in zip(c.index,c['City']):
            vec1,vec2 = text_to_vector(team), text_to_vector(t)
            cos = get_cosine(vec1,vec2)
            if(cos>maxc):
                maxc=cos
                outc=city
        ct.append(outc)
    mlb['City']=ct
    #HandCode some results
    mlb.loc['Oakland Athletics','City']='San Francisco Bay Area'
    return mlb, c

def mlb_correlation(flag=None): 
    # YOUR CODE HERE
    df111,df222= preProcessData3()
    df111['Win_Loss']=df111['Wins'].astype(int)/(df111['Loses'].astype(int)+df111['Wins'].astype(int))
    df111.sort_values('City')
    df222.sort_values('City')
    w_l_mean=[]
    cities=[]
    for group, frame in df111.groupby('City'):
        cities.append(group)
        w_l_mean.append(frame['Win_Loss'].mean())
    #raise NotImplementedError()
    population_by_region = list(map(int,df222['Population'].tolist())) # pass in metropolitan area population from cities
    win_loss_by_region = w_l_mean# pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    if(flag):
        return df111,df222
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [37]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [38]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import Counter
import math

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
WORD = re.compile(r"\w+")
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)
'''text1='Los Angeles Kings'
    text2='Las Vegas'
    ve1=text_to_vector(text1)
    ve2=text_to_vector(text2)
    print(get_cosine(ve1,ve2))'''

def preProcessData4():
    #Basic preProcessing on dataframes
    #NBA Dataframe
    nfl = nfl_df[['team','W','L','year']]
    nfl.where(nfl['year'] == 2018, inplace=True)
    nfl.dropna(inplace=True)
    nfl = nfl[['team','W','L']]
    nfl.columns=['Team','Wins','Loses']
    #mlb['Wins']=mlb['Wins'].str.extract('(\d+)', expand=False)
    nfl.dropna(inplace=True)
    nfl['Team']=nfl['Team'].replace(to_replace ='\*', value = '', regex = True)
    nfl.sort_values(by=['Team'], ascending=True, inplace=True)
    nfl.set_index('Team', inplace=True)
    #Cities Dataframe
    c = cities.drop(['NBA', 'MLB', 'NHL'], axis=1)
    c.columns = ['City','Population','Team']
    c['Team']=c['Team'].replace(to_replace ='\[.*', value = '', regex = True)
    c['Team']=c['Team'].replace("",np.nan)
    c['Team']=c['Team'].replace("—",np.nan)
    c.dropna(inplace=True)
    c.reset_index(drop=True, inplace=True)
    c.drop([13],inplace=True)
    c['Team']=c['City']+' '+c['Team']
    c.set_index('Team', inplace=True)
    c.sort_values(by=['Team'], ascending=True, inplace=True)
    #c = c.drop(['City'],axis=1)
    ct=[]
    for team in nfl.index:
        flag=False
        maxc=0
        outc=np.nan
        for t,city in zip(c.index,c['City']):
            vec1,vec2 = text_to_vector(team), text_to_vector(t)
            cos = get_cosine(vec1,vec2)
            if(cos>maxc):
                maxc=cos
                outc=city
        ct.append(outc)
    nfl['City']=ct
    #HandCode some results
    #nfl.loc['Toronto —','Team']='San Francisco Bay Area'
    return nfl, c

def nfl_correlation(flag = None): 
    # YOUR CODE HERE
    df1111,df2222= preProcessData4()
    df1111.drop(['AFC East','AFC North','AFC South','AFC West','NFC East','NFC North','NFC South','NFC West'], inplace=True)
    df1111['Win_Loss']=df1111['Wins'].astype(int)/(df1111['Loses'].astype(int)+df1111['Wins'].astype(int))
    df1111.sort_values('City')
    df2222.sort_values('City')
    w_l_mean=[]
    cities=[]
    for group, frame in df1111.groupby('City'):
        cities.append(group)
        w_l_mean.append(frame['Win_Loss'].mean())
    #raise NotImplementedError()
    population_by_region = list(map(int,df2222['Population'].tolist())) # pass in metropolitan area population from cities
    win_loss_by_region = w_l_mean# pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    if(flag):
        return df1111, df2222
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [39]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [103]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    nhl_df, c1=nhl_correlation(flag=True)
    mlb_df, c2=mlb_correlation(flag=True)
    nba_df, c3=nba_correlation(flag=True)
    nfl_df, c4=nfl_correlation(flag=True)
    nhl_df.reset_index(inplace=True)
    mlb_df.reset_index(inplace=True)
    nba_df.reset_index(inplace=True)
    nfl_df.reset_index(inplace=True)
    
    nhl_df = nhl_df.groupby('City').mean().reset_index()
    mlb_df = mlb_df.groupby('City').mean().reset_index()
    nba_df = nba_df.groupby('City').mean().reset_index()
    nfl_df = nfl_df.groupby('City').mean().reset_index()
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    
    nhl_nba = pd.merge(nhl_df, nba_df, on='City')
    pval_nhl_nba = stats.ttest_rel(nhl_nba['Win_Loss_x'],nhl_nba['Win_Loss_y'])[1]
    
    nba_mlb = pd.merge(nba_df, mlb_df, on='City')
    pval_nba_mlb = stats.ttest_rel(nba_mlb['Win_Loss_x'],nba_mlb['Win_Loss_y'])[1]
    
    nba_nfl = pd.merge(nba_df, nfl_df, on='City')
    pval_nba_nfl = stats.ttest_rel(nba_nfl['Win_Loss_x'],nba_nfl['Win_Loss_y'])[1]
    
    nhl_mlb = pd.merge(nhl_df, mlb_df, on='City')
    pval_nhl_mlb = stats.ttest_rel(nhl_mlb['Win_Loss_x'],nhl_mlb['Win_Loss_y'])[1]
    
    nhl_nfl = pd.merge(nhl_df, nfl_df, on='City')
    pval_nhl_nfl = stats.ttest_rel(nhl_nfl['Win_Loss_x'],nhl_nfl['Win_Loss_y'])[1]
    
    nfl_mlb = pd.merge(nfl_df, mlb_df, on='City')
    pval_nfl_mlb = stats.ttest_rel(nfl_mlb['Win_Loss_x'],nfl_mlb['Win_Loss_y'])[1]
    
    pvalues = {
       'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_nfl_mlb},
       'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nhl_nba, 'MLB': pval_nba_mlb},
       'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nhl_nba, 'NHL': np.nan, 'MLB': pval_nhl_mlb},
       'MLB': {"NFL": pval_nfl_mlb, 'NBA': pval_nba_mlb, 'NHL': pval_nhl_mlb, 'MLB': np.nan}
      }
    p_values = pd.DataFrame(pvalues)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [104]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
